In [26]:
pathToTestCases = "../test"
pathToBuild = "../build"
pathToTestPre = "../testPre"

alltestfiles = !ls -Sr {pathToTestCases}

In [29]:
class SolveRes:
    def __init__(self, ret, name):
        self.log = ret
        self.name = name
        if(ret[-1].split()[-1] == "UNKNOWN"):
            self.res = "TIMEOUT"
            self.time = "TIMEOUT"
        else:
            self.res = ret[-2].split()[-1]
            self.time = ret[-1].split()[2]
    def prettyPrint(self):
        print("Solved problem:", self.name)
        print("Result:", self.res)
        print("Time:", self.time)

In [4]:
class PresolveOut:
    def __init__(self, preRet, name):
        self.name = name
        self.log = preRet
        try:
            self.time = preRet[-1].split()[2]
        except:
            selt.time = "TIMEOUT"
        try:    
            self.status = preRet[-2].split()[-1]
            self.nrows = preRet[-5].split()[-1]
            self.ncols = preRet[-4].split()[1]
        except:
            self.status = "TIMEOUT"
    def prettyPrint(self):
        print("Presolved problem:", self.name)
        print("Status:", self.status)
        print("Time:", self.time)

In [5]:
 class ComparePreAndBase:
    def __init__(self, res_base, res_withpre, stats_presolver, filename):
        self.name = filename
        self.res_base = res_base
        self.res_withpre = res_withpre
        self.stats_presolver = stats_presolver

In [6]:
def presolve(filename):
    ret = !gtimeout 3600 time {pathToBuild}/PresolveOpb {pathToTestCases}/{filename} {pathToTestPre}/pre.{filename}
    return PresolveOut(ret, filename)

In [7]:
def solve(filename, to_pre_or_not_to_pre):
    filenamestring = filename
    if(to_pre_or_not_to_pre):
        filenamestring = "pre." + filenamestring
        ret = !gtimeout 7200 time {pathToBuild}/roundingsat {pathToTestPre}/{filenamestring}
    else:
        ret = !gtimeout 7200 time {pathToBuild}/roundingsat {pathToTestCases}/{filenamestring}
    return SolveRes(ret, filenamestring)

In [18]:
alltestfiles[:20]

['Generated_notes_20140709_1000_0.05-005.lp.opb',
 'Generated_notes_20140709_1000_0.05-011.lp.opb',
 'Generated_notes_20140709_1000_0.01-000.lp.opb',
 'Generated_notes_20140709_1000_0.01-010.lp.opb',
 'Generated_notes_20140709_1000_0.05-028.lp.opb',
 'Generated_notes_20140709_1000_0.01-019.lp.opb',
 'Generated_notes_20140709_1000_0.01-020.lp.opb',
 'Generated_notes_20140709_1000_0.01-025.lp.opb',
 'Generated_notes_20140709_1000_0.01-003.lp.opb',
 'Generated_notes_20140709_1000_0.01-001.lp.opb',
 'Generated_notes_20140709_1000_0.05-007.lp.opb',
 'Generated_notes_20140709_1000_0.10-014.lp.opb',
 'Generated_notes_20140709_1000_0.01-017.lp.opb',
 'Generated_notes_20140709_1000_0.05-024.lp.opb',
 'Generated_notes_20140709_1000_0.10-004.lp.opb',
 'Generated_notes_20140709_1000_0.01-024.lp.opb',
 'Generated_notes_20140709_1000_0.01-006.lp.opb',
 'Generated_notes_20140709_1000_0.05-004.lp.opb',
 'Generated_notes_20140709_1000_0.05-027.lp.opb',
 'Generated_notes_20140709_1000_0.05-025.lp.opb']

In [9]:
quickProblems = ['air01.0.s.opb',
 'air01.0.u.opb',
 'air02.0.s.opb',
 'air02.0.u.opb',
 'bm23.0.s.opb',
 'bm23.0.u.opb',
 'diamond.0.d.opb',
 'p0040.0.s.opb',
 'p0040.0.u.opb',
 'p0291.0.s.opb',
 'p0291.0.u.opb',
 'pipex.0.s.opb',
 'pipex.0.u.opb',
 'sentoy.0.s.opb',
 'sentoy.0.u.opb',
 'stein15.0.s.opb',
 'stein15.0.u.opb',
 'stein9.0.s.opb',
 'stein9.0.u.opb']




In [27]:
logfile= "temp.log1.json"

In [30]:
import json
i_iteration = 0
with open(logfile, "w") as outfile:
    outfile.write("[\n")

    for prob in quickProblems:
        print("i = " + str(i_iteration))
        i_iteration += 1
        jsonobj = {"name": prob }
        
        presolveinf = presolve(prob)
        presolveinf.prettyPrint()
        jsonobj['status'] = presolveinf.status
        jsonobj['presolveTime'] = float(presolveinf.time)

        if(presolveinf.status == "TIMEOUT"):
            for line in presolveinf.log:
                print(" " + line)

        if(presolveinf.status == "1"):
            solveres = solve(prob, False)
            print(" - solver time:       " + solveres.time)
            #jsonobj['solverBaseTime'] = float(solveres.time)
            #jsonobj['solverBaseLog'] = solveres.log
            
            
            solverespre = solve(prob, True)
            print(" - solver time (pre): "+ solverespre.time)
            print(" - same result:", solveres.res == solverespre.res)
            #jsonobj['solverPreTime'] = float(solverespre.time)
            #jsonobj['solverPreLog'] = solveres.log
            #jsonobj['sameRes'] = str(solveres.res == solverespre.res)
            
        #timediff = float(solveres.time[:-1]) - float(solverespre.time[:-1]) - float(presolveinf.time[:-1])
        outfile.write(json.dumps(jsonobj)+",\n")


        #print("solver time: " + solveres.time + "   solver time (pre): "+ solverespre.time)
        print()
    outfile.write("\n]")
    

i = 0
Presolved problem: air01.0.s.opb
Status: 1
Time: 0.66
 - solver time:       0.13
 - solver time (pre): 0.04
 - same result: True

i = 1
Presolved problem: air01.0.u.opb
Status: 1
Time: 0.47
 - solver time:       0.34
 - solver time (pre): 0.15
 - same result: True

i = 2
Presolved problem: air02.0.s.opb
Status: 1
Time: 73.37
 - solver time:       1.57
 - solver time (pre): 0.89
 - same result: True

i = 3
Presolved problem: air02.0.u.opb
Status: 1
Time: 75.16
 - solver time:       2.84
 - solver time (pre): 1.90
 - same result: True

i = 4
Presolved problem: bm23.0.s.opb
Status: 0
Time: 0.01

i = 5
Presolved problem: bm23.0.u.opb
Status: 0
Time: 0.01

i = 6
Presolved problem: diamond.0.d.opb
Status: 4
Time: 0.00

i = 7
Presolved problem: p0040.0.s.opb
Status: 1
Time: 0.01
 - solver time:       0.03
 - solver time (pre): 0.00
 - same result: True

i = 8
Presolved problem: p0040.0.u.opb
Status: 1
Time: 0.01
 - solver time:       0.03
 - solver time (pre): 0.00
 - same result: True


NameError: name 'ret' is not defined

In [11]:
import json

with open(logfile, 'w') as outfile:
    outfile.write("[\n")
    for prob in alltestOpt:
        #solveres = solve(prob, False)
        #outfile.write(json.dumps(solveres.__dict__)+',\n')
        
        presolveout = presolve(prob)
        outfile.write(json.dumps(presolveout.__dict__)+',\n')
        
        #presolveres = solve(prob, True)
        #outfile.write(json.dumps(presolveres.__dict__)+',\n')
        
        print(prob)
        #print("Same result: "   + str(solveres.res == presolveres.res))
        print("Presolve time: " + presolveout.time)
        #print("solver res: "    + solveres.res  + "   solver res (pre): "  + presolveres.res)
        #print("solver time: "   + solveres.time + "   solver time (pre): " + presolveres.time)
        print()
    outfile.write("]")

air01.opb
Presolve time: 0.09

air02.opb
Presolve time: 3.01

air03.opb
Presolve time: 3.42

air04.opb
Presolve time: 1.55

air05.opb
Presolve time: 0.97

air06.opb
Presolve time: 1.32

bm23.opb
Presolve time: 0.01

cracpb1.opb
Presolve time: 0.07

diamond.opb
Presolve time: 0.00

enigma.opb
Presolve time: 0.01

l152lav.opb
Presolve time: 0.17

lp4l.opb
Presolve time: 0.07

lseu.opb
Presolve time: 0.01

mod008.opb
Presolve time: 0.01

mod010.opb
Presolve time: 0.27

normalized-single-obj-f47-DC-Side1.seq-B-2-1-EDCBAir.opb
Presolve time: 0.00

p0033.opb
Presolve time: 0.00

p0040.opb
Presolve time: 0.01

p0201.opb
Presolve time: 0.02

p0282.opb
Presolve time: 0.03

p0291.opb
Presolve time: 0.02

p0548.opb
Presolve time: 0.04

p2756.opb
Presolve time: 0.18

p6000.opb
Presolve time: 0.77

pipex.opb
Presolve time: 0.01

sentoy.opb
Presolve time: 0.02

stein15.opb
Presolve time: 0.00

stein27.opb
Presolve time: 0.01

stein45.opb
Presolve time: 0.01

stein9.opb
Presolve time: 0.00



In [10]:
pathToTestCases = "../testOpt"
pathToBuild = "../build"
pathToTestPre = "../testPre"

alltestfolders = !ls {pathToTestCases}

In [12]:
for folder in alltestfolders:
    #Presolve all
    prob = pathToTestCases +"/"+ folder + "/" + folder + ".opb"
    !{pathToBuild}/PresolveOpb -I {prob}

Filename in given: ../testOpt/air01/air01.opb Filename out given: ../testOpt/air01/air01.pre.opb
Parsing ...
  Parser found: 771 vars
  Parser found: 23 constraints
Done!

starting presolve of problem ../testOpt/air01/air01.opb:
  rows:     23
  columns:  771 (771 int., 0 cont.)
  nonzeros: 4215

round 0   ( Trivial  ):    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (   Fast   ):   11 del cols,    0 del rows,    0 chg bounds,   11 chg sides,   11 chg coeffs,   11 tsx applied,    1 tsx conflicts
round 2   (  Medium  ):   18 del cols,    0 del rows,    7 chg bounds,   11 chg sides,   11 chg coeffs,   19 tsx applied,    1 tsx conflicts
round 3   (Exhaustive):  363 del cols,    0 del rows,  352 chg bounds,   11 chg sides,   11 chg coeffs,  913 tsx applied,  346 tsx conflicts
round 4   (Exhaustive): Unchanged
activating delayed presolvers
round 5   (  Final   ): Unchanged
presolved 5 rounds:  363 del cols,    0 

Status code of test problem: 1
Filename in given: ../testOpt/air04/air04.opb Filename out given: ../testOpt/air04/air04.pre.opb
Parsing ...
  Parser found: 8904 vars
  Parser found: 823 constraints
Done!

starting presolve of problem ../testOpt/air04/air04.opb:
  rows:     823
  columns:  8904 (8904 int., 0 cont.)
  nonzeros: 72965

round 0   ( Trivial  ):    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (  Medium  ):    1 del cols,    1 del rows,    0 chg bounds,    0 chg sides,    8 chg coeffs,    1 tsx applied,    1 tsx conflicts
round 2   (   Fast   ):   30 del cols,    3 del rows,   29 chg bounds,    0 chg sides,    8 chg coeffs,   37 tsx applied,    1 tsx conflicts
round 3   (  Medium  ):   31 del cols,    4 del rows,   29 chg bounds,    0 chg sides,   14 chg coeffs,   38 tsx applied,    1 tsx conflicts
round 4   (Exhaustive):   35 del cols,    4 del rows,   33 chg bounds,    0 chg sides,   14 chg coeff

Status code of test problem: 1
Filename in given: ../testOpt/air06/air06.opb Filename out given: ../testOpt/air06/air06.pre.opb
Parsing ...
  Parser found: 8627 vars
  Parser found: 825 constraints
Done!

starting presolve of problem ../testOpt/air06/air06.opb:
  rows:     825
  columns:  8627 (8627 int., 0 cont.)
  nonzeros: 70806

round 0   ( Trivial  ):    3 del cols,    5 del rows,    3 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (   Fast   ):   55 del cols,   17 del rows,   55 chg bounds,    0 chg sides,    0 chg coeffs,   99 tsx applied,    0 tsx conflicts
round 2   (  Medium  ):   58 del cols,   22 del rows,   56 chg bounds,    0 chg sides,   24 chg coeffs,  101 tsx applied,    2 tsx conflicts
round 3   (   Fast   ):   67 del cols,   25 del rows,   65 chg bounds,    0 chg sides,   24 chg coeffs,  112 tsx applied,    2 tsx conflicts
round 4   (Exhaustive):   69 del cols,   25 del rows,   67 chg bounds,    0 chg sides,   24 chg coeff

Filename in given: ../testOpt/cracpb1/cracpb1.opb Filename out given: ../testOpt/cracpb1/cracpb1.pre.opb
Parsing ...
  Parser found: 572 vars
  Parser found: 144 constraints
Done!

starting presolve of problem ../testOpt/cracpb1/cracpb1.opb:
  rows:     144
  columns:  572 (572 int., 0 cont.)
  nonzeros: 4676

round 0   ( Trivial  ):    0 del cols,   18 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (   Fast   ):   54 del cols,   18 del rows,    0 chg bounds,   54 chg sides,   54 chg coeffs,   54 tsx applied,    0 tsx conflicts
round 2   (Exhaustive): Unchanged
activating delayed presolvers
round 3   (Exhaustive):   54 del cols,   18 del rows,    0 chg bounds,   54 chg sides, 1339 chg coeffs,   89 tsx applied,    4 tsx conflicts
round 4   (Exhaustive):   56 del cols,   18 del rows,    2 chg bounds,   54 chg sides, 1339 chg coeffs,   91 tsx applied,    6 tsx conflicts
round 5   (  Final   ): Unchanged
presolved 5 rounds:   56 d

Filename in given: ../testOpt/l152lav/l152lav.opb Filename out given: ../testOpt/l152lav/l152lav.pre.opb
Parsing ...
  Parser found: 1989 vars
  Parser found: 97 constraints
Done!

starting presolve of problem ../testOpt/l152lav/l152lav.opb:
  rows:     97
  columns:  1989 (1989 int., 0 cont.)
  nonzeros: 9922

round 0   ( Trivial  ):    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (Exhaustive): Unchanged
activating delayed presolvers
round 2   (Exhaustive):    0 del cols,    0 del rows,    0 chg bounds,   42 chg sides,  670 chg coeffs,   94 tsx applied,    1 tsx conflicts
round 3   (   Fast   ):    3 del cols,    0 del rows,    0 chg bounds,   45 chg sides,  673 chg coeffs,   97 tsx applied,    1 tsx conflicts
round 4   (Exhaustive):    4 del cols,    1 del rows,    0 chg bounds,   45 chg sides,  703 chg coeffs,   98 tsx applied,    1 tsx conflicts
round 5   (   Fast   ):    5 del cols,    1 del rows,    0 

...

Filename in given: ../testOpt/p0201/p0201.opb Filename out given: ../testOpt/p0201/p0201.pre.opb
Parsing ...
  Parser found: 201 vars
  Parser found: 133 constraints
Done!

starting presolve of problem ../testOpt/p0201/p0201.opb:
  rows:     133
  columns:  201 (201 int., 0 cont.)
  nonzeros: 1923

round 0   ( Trivial  ):    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (   Fast   ):    6 del cols,    0 del rows,    6 chg bounds,    0 chg sides,   72 chg coeffs,   36 tsx applied,    0 tsx conflicts
round 2   (   Fast   ):    6 del cols,    0 del rows,    6 chg bounds,    3 chg sides,   78 chg coeffs,   39 tsx applied,    0 tsx conflicts
round 3   (  Medium  ):    6 del cols,    0 del rows,    6 chg bounds,    6 chg sides,   91 chg coeffs,   52 tsx applied,    0 tsx conflicts
round 4   (Exhaustive): Unchanged
activating delayed presolvers
round 5   (  Final   ): Unchanged
presolved 5 rounds:    6 del cols,    

Filename in given: ../testOpt/p0548/p0548.opb Filename out given: ../testOpt/p0548/p0548.pre.opb
Parsing ...
  Parser found: 548 vars
  Parser found: 176 constraints
Done!

starting presolve of problem ../testOpt/p0548/p0548.opb:
  rows:     176
  columns:  548 (548 int., 0 cont.)
  nonzeros: 1711

round 0   ( Trivial  ):    2 del cols,   12 del rows,    2 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (   Fast   ):   31 del cols,   22 del rows,   18 chg bounds,   57 chg sides,  163 chg coeffs,  107 tsx applied,    0 tsx conflicts
round 2   (   Fast   ):   62 del cols,   24 del rows,   49 chg bounds,   60 chg sides,  185 chg coeffs,  145 tsx applied,    0 tsx conflicts
round 3   (   Fast   ):   71 del cols,   25 del rows,   58 chg bounds,   60 chg sides,  189 chg coeffs,  158 tsx applied,    0 tsx conflicts
round 4   (   Fast   ):   71 del cols,   25 del rows,   58 chg bounds,   60 chg sides,  192 chg coeffs,  161 tsx applied,    0 tsx confl

round 19  (  Medium  ):  206 del cols,   71 del rows,  169 chg bounds,  206 chg sides,  568 chg coeffs,  580 tsx applied,    1 tsx conflicts
round 20  (Exhaustive):  206 del cols,   71 del rows,  169 chg bounds,  206 chg sides,  574 chg coeffs,  586 tsx applied,    1 tsx conflicts
round 21  (  Final   ): Unchanged
presolved 21 rounds:  206 del cols,   71 del rows,  169 chg bounds,  206 chg sides,  574 chg coeffs,  586 tsx applied,    1 tsx conflicts

          presolver     nb calls   success calls(%)    nb transactions     tsx applied(%)  execution time(s) 
       colsingleton           21               38.1                 74              100.0              0.000
    coefftightening           21               52.4                420               99.8              0.001
        propagation           21                9.5                 17              100.0              0.001
      simpleprobing            3                0.0                  0                0.0              0.000

Filename in given: ../testOpt/sentoy/sentoy.opb Filename out given: ../testOpt/sentoy/sentoy.pre.opb
Parsing ...
  Parser found: 60 vars
  Parser found: 30 constraints
Done!

starting presolve of problem ../testOpt/sentoy/sentoy.opb:
  rows:     30
  columns:  60 (60 int., 0 cont.)
  nonzeros: 1800

round 0   ( Trivial  ):    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (Exhaustive): Unchanged
activating delayed presolvers
round 2   (  Final   ): Unchanged
presolved 2 rounds:    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts

          presolver     nb calls   success calls(%)    nb transactions     tsx applied(%)  execution time(s) 
       colsingleton            2                0.0                  0                0.0              0.000
    coefftightening            2                0.0                  0                0.0              

Filename in given: ../testOpt/stein45/stein45.opb Filename out given: ../testOpt/stein45/stein45.pre.opb
Parsing ...
  Parser found: 45 vars
  Parser found: 331 constraints
Done!

starting presolve of problem ../testOpt/stein45/stein45.opb:
  rows:     331
  columns:  45 (45 int., 0 cont.)
  nonzeros: 1034

round 0   ( Trivial  ):    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts
round 1   (Exhaustive): Unchanged
activating delayed presolvers
round 2   (  Final   ): Unchanged
presolved 2 rounds:    0 del cols,    0 del rows,    0 chg bounds,    0 chg sides,    0 chg coeffs,    0 tsx applied,    0 tsx conflicts

          presolver     nb calls   success calls(%)    nb transactions     tsx applied(%)  execution time(s) 
       colsingleton            2                0.0                  0                0.0              0.000
    coefftightening            2                0.0                  0                0.0      

In [17]:
#get pre solution
for folder in alltestfolders:
    if (folder[0] == 'a' and folder[4] != '5' and folder[4] != '6'):
        prob = pathToTestCases + "/" + folder + "/" + folder
        !{pathToBuild}/roundingsat {prob}.pre.opb --print-sol=1 >> {prob}.pre.solution

^C
^C
^C
